In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
print(os.getcwd())

c:\Users\utcpret\Documents\NF26\nf26_project


In [2]:
!pip install geopy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\utcpret\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# Initialize regular pandas and other basic libraries
import pandas as pd
import numpy as np
from datetime import datetime, date

# Initialize Spark session
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pyspark.pandas  as ps

# Create SparkSession first
spark = SparkSession.builder \
    .appName("NF26_Project") \
    .getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")

# Check PySpark version
print(f"PySpark version: {spark.version}")

# Import ETL functions - these don't depend on pyspark.pandas
from etl_warehouse import *

# Define a helper function to read CSV with pandas and convert to Spark DataFrame
def read_csv_to_spark(file_path, sep=';', encoding='utf-8', index_col=None):
    """Read CSV with pandas and convert to Spark DataFrame"""
    pdf = pd.read_csv(file_path, sep=sep, encoding=encoding)
    if index_col:
        pdf.set_index(index_col, inplace=True)
    return spark.createDataFrame(pdf)

PySpark version: 3.5.5


In [4]:
spark = SparkSession.builder \
    .appName("NF26_Project") \
    .getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")

In [5]:
## Définition du fuseau horaire sur paris ##

spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")


Distance

In [6]:
import os
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable, GeocoderServiceError
import time

DISTANCE_CACHE_FILE = "distances_cache.csv"
COORDINATES_CACHE_FILE = "coordinates_cache.csv"

def calculate_distance(departure_city, destination_city, geolocator, max_retries=20):

    # Geocode with retry
    def geocode_with_retry(query):
        for _ in range(max_retries):
            try:
                return geolocator.geocode(query)
            except (GeocoderTimedOut, GeocoderUnavailable, GeocoderServiceError):
                time.sleep(0.5)
            except Exception:
                break
        return None

    # Load caches
    if os.path.exists(DISTANCE_CACHE_FILE):
        distance_cache_df = pd.read_csv(DISTANCE_CACHE_FILE)
    else:
        distance_cache_df = pd.DataFrame(columns=["departure", "destination", "distance_km"])
    
    if os.path.exists(COORDINATES_CACHE_FILE):
        coordinates_cache_df = pd.read_csv(COORDINATES_CACHE_FILE)
    else:
        coordinates_cache_df = pd.DataFrame(columns=["city", "latitude", "longitude"])

    # Check if distance already cached
    cached_row = distance_cache_df[
        ((distance_cache_df["departure"] == departure_city) & (distance_cache_df["destination"] == destination_city)) |
        ((distance_cache_df["departure"] == destination_city) & (distance_cache_df["destination"] == departure_city))
    ]
    if not cached_row.empty:
        return float(cached_row.iloc[0]["distance_km"])

    # Get or fetch coordinates for departure
    cached_departure_city = coordinates_cache_df[coordinates_cache_df["city"] == departure_city]
    if not cached_departure_city.empty:
        departure_coords = (
            float(cached_departure_city.iloc[0]["latitude"]),
            float(cached_departure_city.iloc[0]["longitude"])
        )
    else:
        departure_location = geocode_with_retry(departure_city)
        if departure_location is None:
            return None
        departure_coords = (departure_location.latitude, departure_location.longitude)
        new_row = pd.DataFrame([{
            "city": departure_city,
            "latitude": departure_coords[0],
            "longitude": departure_coords[1]
        }])
        coordinates_cache_df = pd.concat([coordinates_cache_df, new_row], ignore_index=True)
        coordinates_cache_df.to_csv(COORDINATES_CACHE_FILE, index=False)

    # Get or fetch coordinates for destination
    cached_destination_city = coordinates_cache_df[coordinates_cache_df["city"] == destination_city]
    if not cached_destination_city.empty:
        destination_coords = (
            float(cached_destination_city.iloc[0]["latitude"]),
            float(cached_destination_city.iloc[0]["longitude"])
        )
    else:
        destination_location = geocode_with_retry(destination_city)
        if destination_location is None:
            return None
        destination_coords = (destination_location.latitude, destination_location.longitude)
        new_row = pd.DataFrame([{
            "city": destination_city,
            "latitude": destination_coords[0],
            "longitude": destination_coords[1]
        }])
        coordinates_cache_df = pd.concat([coordinates_cache_df, new_row], ignore_index=True)
        coordinates_cache_df.to_csv(COORDINATES_CACHE_FILE, index=False)

    # Calculate and save distance
    distance_km = geodesic(departure_coords, destination_coords).km
    new_row = pd.DataFrame([{
        "departure": departure_city,
        "destination": destination_city,
        "distance_km": distance_km
    }])
    distance_cache_df = pd.concat([distance_cache_df, new_row], ignore_index=True)
    distance_cache_df.to_csv(DISTANCE_CACHE_FILE, index=False)

    return distance_km


Matériel GES

In [7]:
from pyspark.sql import functions as F
import pandas as pd

pdf_informatique = pd.read_csv('BDD_BGES/BDD_BGES/materiel_informatique_impact.csv', sep=',', encoding='utf-8')
moyenne_ges = pdf_informatique["Impact"].mean()

def materiel_informatique_GES(type_mat, modele_mat):
    if modele_mat != "modèle par défaut" and modele_mat.strip() != "":
        match = pdf_informatique[pdf_informatique["Modèle"] == modele_mat]
        if not match.empty:
            return match.iloc[0]["Impact"]
    else:
        match = pdf_informatique[
            (pdf_informatique["Type"] == type_mat) &
            (pdf_informatique["Modèle"] == "modèle par défaut")
        ]
        if not match.empty:
            return match.iloc[0]["Impact"]
        else:
            return moyenne_ges
    return None

def tranform_with_GES(df_materiel):
    rows_with_ges = []
    for _, row in df_materiel.iterrows():
        ges_value = materiel_informatique_GES(row["TYPE"], row["MODELE"])
        if ges_value is not None:
            ges_value *= 0.001
            row_dict = row.to_dict()
            row_dict["GES"] = ges_value
            rows_with_ges.append(row_dict)
    if rows_with_ges:
        return pd.DataFrame(rows_with_ges)
    else:
        return df_materiel


Mission GES

In [8]:


list_with_no_avion = ["Compiègne"]
geolocator = Nominatim(user_agent="mission_distance_calculator")

def get_closest_city_with_avion(ville_destination):
    closest_city_with_avion = {"Compiègne": "Paris"}
    return closest_city_with_avion.get(ville_destination, ville_destination)

def avion_GES(ville_depart, ville_destination, aller_retour):
    distance = calculate_distance(ville_depart, ville_destination, geolocator)
    if distance is None:
        return 0
    distance += 95
    if distance < 1000:
        facteur_emissions = 0.2586
    elif distance < 3500:
        facteur_emissions = 0.1875
    else:
        facteur_emissions = 0.152
    GES = distance * facteur_emissions
    if aller_retour == "oui":
        GES = GES * 2

    return GES

def taxi_GES(ville_depart, ville_destination, aller_retour):
    if ville_depart == ville_destination:
        distance = 5
    else:
        distance = calculate_distance(ville_depart, ville_destination, geolocator)
    facteur_emissions = 0.2156
    GES = distance * 1.2 * 2 * facteur_emissions
    if aller_retour == "oui":
        GES = GES * 2
    
    return GES

def commun_GES(ville_depart, ville_destination, aller_retour):
    if ville_depart == ville_destination:
        distance = 5
    else:
        distance = calculate_distance(ville_depart, ville_destination, geolocator)
    #facteur d'emission = moyenne entre Tramway > 250 000 habitants + rer + Bus > 250 000 habitants
    facteur_emissions = 0.047
    #coeff_multiplicateur = moyenne entre 1,2 pour le RER, 1,7 pour le métro et 1,5 pour le bus
    coeff_multiplicateur = (1.2 + 1.7 + 1.5) / 3
    GES = distance * coeff_multiplicateur * facteur_emissions
    if aller_retour == "oui":
        GES = GES * 2

    return GES

def train_GES(ville_depart, ville_destination, aller_retour):
    if ville_depart == ville_destination:
        distance = 5
    else :
        distance = calculate_distance(ville_depart, ville_destination, geolocator)
    if distance < 200:
        facteur_emissions = 0.018
    else:
        facteur_emissions = 0.0033
    GES = distance * 1.2 * facteur_emissions
    if aller_retour == "oui":
        GES = GES * 2

    return GES

def mission_bilan_carbone(df_mission):
    import pandas as pd
    rows_with_ges = []
    for _, row in df_mission.iterrows():
        print("mission numero :", _, "faite")
        transport = row["TRANSPORT"]
        ville_depart = row["VILLE_DEPART"]
        ville_destination = row["VILLE_DESTINATION"]
        aller_retour = row["ALLER_RETOUR"]
        GES = None
        if transport == "Avion":
            if ville_destination in list_with_no_avion:
                ville_avion = get_closest_city_with_avion(ville_destination)
                GES = train_GES(ville_destination, ville_avion, aller_retour)
                GES = avion_GES(ville_depart, ville_avion, aller_retour)
            else:
                GES = avion_GES(ville_depart, ville_destination, aller_retour)
        elif transport == "Taxi":
            GES = taxi_GES(ville_depart, ville_destination, aller_retour)
        elif transport == "Transports en commun":
            GES = commun_GES(ville_depart, ville_destination, aller_retour)
        elif transport == "Train":
            GES = train_GES(ville_depart, ville_destination, aller_retour)
        if GES is not None:
            GES *= 0.001
            rows_with_ges.append({**row.to_dict(), "GES": GES})
        else:
            print("erreur pour row = ", row)
    if rows_with_ges:
        return pd.DataFrame(rows_with_ges)
    else:
        return df_mission


ETL_warehouse

In [9]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import glob
import pandas as pd
import pytz
from datetime import datetime, timedelta
from pyspark.sql import SparkSession


# Configuration des chemins
SOURCE_PATH = "BDD_BGES\\BDD_BGES"
DW_PATH = "DATA_WAREHOUSE"  # Chemin vers le Data Warehouse

# Créer les répertoires DW s'ils n'existent pas
os.makedirs(DW_PATH, exist_ok=True)
os.makedirs(os.path.join(DW_PATH, "FAITS_MISSION"), exist_ok=True)
os.makedirs(os.path.join(DW_PATH, "FAITS_MATERIEL"), exist_ok=True)
os.makedirs(os.path.join(DW_PATH, "DIM_DATE"), exist_ok=True)
os.makedirs(os.path.join(DW_PATH, "DIM_MISSION"), exist_ok=True)
os.makedirs(os.path.join(DW_PATH, "DIM_MATERIEL"), exist_ok=True)

# Mappage des villes vers leurs fuseaux horaires
FUSEAUX_VILLES = {
    "BERLIN": "Europe/Berlin",
    "PARIS": "Europe/Paris",
    "LONDON": "Europe/London",
    "LOSANGELES": "America/Los_Angeles",
    "NEWYORK": "America/New_York",
    "SHANGHAI": "Asia/Shanghai",
}

def table_existe(table_path):
    """Vérifie si une table existe déjà"""
    return os.path.exists(table_path)

def convertir_date_fuseau_paris(date_original, ville_source):
    """Convertit une date du fuseau horaire source au fuseau horaire de Paris"""
    # Obtenir le fuseau horaire source
    tz_source = pytz.timezone(FUSEAUX_VILLES.get(ville_source, "UTC"))
    # Obtenir le fuseau horaire de Paris
    tz_paris = pytz.timezone("Europe/Paris")
    
    # Localiser la date dans son fuseau d'origine puis la convertir en fuseau Paris
    date_localisee = tz_source.localize(pd.to_datetime(date_original), is_dst=None)
    date_paris = date_localisee.astimezone(tz_paris)
    
    return date_paris

def chercher_fichiers_avec_conversion_fuseau(date_str, type_donnees):
    """Recherche les fichiers correspondant à une date exacte"""
    # On ne considère que la date exacte spécifiée, plus les jours adjacents
    dates_a_verifier = [date_str]
    
    fichiers_trouves = []
    
    for date_check in dates_a_verifier:
        for ville in FUSEAUX_VILLES.keys():
            if type_donnees == "MISSION":
                fichier_path = f"{SOURCE_PATH}\\BDD_BGES_{ville}\\BDD_BGES_{ville}_MISSION\\MISSION_{date_check}.txt"
            else:  # MATERIEL_INFORMATIQUE
                fichier_path = f"{SOURCE_PATH}\\BDD_BGES_{ville}\\BDD_BGES_{ville}_INFORMATIQUE\\MATERIEL_INFORMATIQUE_{date_check}.txt"
                
            if os.path.exists(fichier_path):
                fichiers_trouves.append((fichier_path, ville, date_check))
    
    return fichiers_trouves

def lire_fichiers_source(date_str, type_donnees):
    """Lit les fichiers sources en tenant compte des fuseaux horaires"""
    fichiers_trouves = chercher_fichiers_avec_conversion_fuseau(date_str, type_donnees)
    
    if not fichiers_trouves:
        raise FileNotFoundError(f"Aucune donnée trouvée pour {type_donnees} à la date {date_str}")
    
    all_dfs = []
    date_paris_cible = pd.to_datetime(f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}")
    
    for fichier_path, ville, date_fichier in fichiers_trouves:
        try:
            df = pd.read_csv(fichier_path, sep=';')
            df['VILLE_SOURCE'] = ville
            df['DATE_FICHIER'] = date_fichier
            
            # Normaliser les dates selon le fuseau horaire
            col_date = 'DATE_MISSION' if type_donnees == 'MISSION' else 'DATE_ACHAT'
            df[col_date] = pd.to_datetime(df[col_date])
            
            # Créer une colonne avec la date convertie au fuseau de Paris
            df['DATE_PARIS'] = df.apply(
                lambda row: convertir_date_fuseau_paris(row[col_date], row['VILLE_SOURCE']), 
                axis=1
            )
            
            # Filtrer pour ne garder que les lignes dont la date Paris correspond à la date cible
            df_filtered = df[df['DATE_PARIS'].dt.date == date_paris_cible.date()]
            
            if not df_filtered.empty:
                all_dfs.append(df_filtered)
                print(f"Fichier chargé et filtré: {fichier_path} - {len(df_filtered)} lignes")
            
        except Exception as e:
            print(f"Erreur lors du chargement {fichier_path}: {e}")
    
    if not all_dfs:
        raise FileNotFoundError(f"Aucune donnée correspondant à la date {date_str} après conversion des fuseaux horaires")
    
    # Combiner tous les dataframes
    return pd.concat(all_dfs, ignore_index=True)

def creer_dimension_date(dates_paris):
    """Crée ou met à jour la table dimension date"""
    dimension_date_path = os.path.join(DW_PATH, "DIM_DATE", "dimension_date.csv")
    
    # Créer une liste de dates uniques
    dates_uniques = pd.DataFrame({'DATE_PARIS': pd.Series(dates_paris).dt.date.unique()})
    dates_uniques = pd.DataFrame(dates_uniques['DATE_PARIS'])
    
    # Enrichir avec des attributs de date
    dates_uniques['KeyDate'] = dates_uniques['DATE_PARIS'].apply(lambda x: x.strftime('%Y%m%d'))
    dates_uniques['ANNEE'] = dates_uniques['DATE_PARIS'].apply(lambda x: x.year)
    dates_uniques['MOIS'] = dates_uniques['DATE_PARIS'].apply(lambda x: x.month)
    dates_uniques['JOUR'] = dates_uniques['DATE_PARIS'].apply(lambda x: x.day)
    dates_uniques['JOUR_SEMAINE'] = dates_uniques['DATE_PARIS'].apply(lambda x: x.weekday())
    dates_uniques['NOM_JOUR'] = dates_uniques['DATE_PARIS'].apply(lambda x: x.strftime('%A'))  # Nom du jour
    dates_uniques['TRIMESTRE'] = dates_uniques['DATE_PARIS'].apply(lambda x: (x.month - 1) // 3 + 1)
    
    # Si la table existe déjà, fusionner avec de nouvelles dates
    if table_existe(dimension_date_path):
        dim_date_existante = pd.read_csv(dimension_date_path)
        dim_date_existante['DATE_PARIS'] = pd.to_datetime(dim_date_existante['DATE_PARIS']).dt.date
        
        # Identifier les nouvelles dates
        dates_existantes = set(dim_date_existante['KeyDate'])
        nouvelles_dates = dates_uniques[~dates_uniques['KeyDate'].isin(dates_existantes)]
        
        # Fusionner si de nouvelles dates existent
        if len(nouvelles_dates) > 0:
            dim_date_maj = pd.concat([dim_date_existante, nouvelles_dates], ignore_index=True)
            dim_date_maj.to_csv(dimension_date_path, index=False)
            print(f"Table dimension DATE mise à jour avec {len(nouvelles_dates)} nouvelles dates")
    else:
        # Créer la table
        dates_uniques.to_csv(dimension_date_path, index=False)
        print(f"Table dimension DATE créée avec {len(dates_uniques)} dates")
    
    return dates_uniques

def creer_dimension_mission(df_missions):
    """Crée ou met à jour la table dimension mission"""
    dimension_mission_path = os.path.join(DW_PATH, "DIM_MISSION", "dimension_mission.csv")
    
    # Colonnes à conserver pour la dimension
    cols_mission = ['ID_MISSION', 'TYPE_MISSION', 'VILLE_DEPART', 'PAYS_DEPART', 
                   'VILLE_DESTINATION', 'PAYS_DESTINATION', 'TRANSPORT', 'ALLER_RETOUR']
    dim_mission_nouvelles = df_missions[cols_mission].drop_duplicates()
    
    # Si la table existe déjà, fusionner avec de nouvelles missions
    if table_existe(dimension_mission_path):
        dim_mission_existante = pd.read_csv(dimension_mission_path)
        
        # Identifier les nouvelles missions
        missions_existantes = set(dim_mission_existante['ID_MISSION'])
        nouvelles_missions = dim_mission_nouvelles[~dim_mission_nouvelles['ID_MISSION'].isin(missions_existantes)]
        
        # Fusionner si de nouvelles missions existent
        if len(nouvelles_missions) > 0:
            nouvelles_missions = mission_bilan_carbone(nouvelles_missions)
            dim_mission_maj = pd.concat([dim_mission_existante, nouvelles_missions], ignore_index=True)
            dim_mission_maj.to_csv(dimension_mission_path, index=False)
            print(f"Table dimension MISSION mise à jour avec {len(nouvelles_missions)} nouvelles missions")
    else:
        # Créer la table        
        dim_mission_nouvelles = mission_bilan_carbone(dim_mission_nouvelles)
        dim_mission_nouvelles.to_csv(dimension_mission_path, index=False)
        print(f"Table dimension MISSION créée avec {len(dim_mission_nouvelles)} missions")
    
    return dim_mission_nouvelles

def creer_dimension_materiel(df_materiel):
    """Crée ou met à jour la table dimension matériel"""
    dimension_materiel_path = os.path.join(DW_PATH, "DIM_MATERIEL", "dimension_materiel.csv")
    
    # Colonnes à conserver pour la dimension
    cols_materiel = ['ID_MATERIELINFO', 'TYPE', 'MODELE']
    dim_materiel_nouvelles = df_materiel[cols_materiel].drop_duplicates()
    
    # Si la table existe déjà, fusionner avec de nouveaux matériels
    if table_existe(dimension_materiel_path):
        dim_materiel_existante = pd.read_csv(dimension_materiel_path)
        
        # Identifier les nouveaux matériels
        materiels_existants = set(dim_materiel_existante['ID_MATERIELINFO'])
        nouveaux_materiels = dim_materiel_nouvelles[~dim_materiel_nouvelles['ID_MATERIELINFO'].isin(materiels_existants)]
        
        # Fusionner si de nouveaux matériels existent
        if len(nouveaux_materiels) > 0:
            nouveaux_materiels = tranform_with_GES(nouveaux_materiels)
            dim_materiel_maj = pd.concat([dim_materiel_existante, nouveaux_materiels], ignore_index=True)
            dim_materiel_maj.to_csv(dimension_materiel_path, index=False)
            print(f"Table dimension MATERIEL mise à jour avec {len(nouveaux_materiels)} nouveaux matériels")
    else:
        # Créer la table
        dim_materiel_nouvelles = tranform_with_GES(dim_materiel_nouvelles)
        dim_materiel_nouvelles.to_csv(dimension_materiel_path, index=False)
        print(f"Table dimension MATERIEL créée avec {len(dim_materiel_nouvelles)} matériels")
    
    return dim_materiel_nouvelles

# Modifiez les fonctions créer_table_fait_mission et créer_table_fait_materiel

def creer_table_fait_mission(date_str):
    """Crée ou met à jour la table de faits mission"""
    output_path = os.path.join(DW_PATH, "FAITS_MISSION", "faits_mission.csv")
    
    # Extraire les données
    try:
        df_missions = lire_fichiers_source(date_str, "MISSION")
        
        # Créer les dimensions associées
        creer_dimension_date(df_missions['DATE_PARIS'])
        creer_dimension_mission(df_missions)
        
        # Créer la table de faits simplifiée
        df_missions['KeyDate'] = df_missions['DATE_PARIS'].dt.strftime('%Y%m%d')
        faits_mission = df_missions[['ID_MISSION', 'ID_PERSONNEL', 'KeyDate']].copy()
        
        # Convertir explicitement toutes les colonnes en string
        faits_mission['ID_MISSION'] = faits_mission['ID_MISSION'].astype(str)
        faits_mission['ID_PERSONNEL'] = faits_mission['ID_PERSONNEL'].astype(str)
        faits_mission['KeyDate'] = faits_mission['KeyDate'].astype(str)
        
        # Si la table existe déjà, fusionner avec de nouvelles lignes
        if table_existe(output_path):
            print(f"La table de faits MISSION existe déjà, mise à jour...")
            df_existant = pd.read_csv(output_path)
            
            # Convertir explicitement toutes les colonnes en string
            df_existant['ID_MISSION'] = df_existant['ID_MISSION'].astype(str)
            df_existant['ID_PERSONNEL'] = df_existant['ID_PERSONNEL'].astype(str)
            df_existant['KeyDate'] = df_existant['KeyDate'].astype(str)
            
            # Créer une clé composite pour identifier les lignes uniques
            df_existant['COMPOSITE_KEY'] = df_existant['ID_MISSION'] + '_' + df_existant['ID_PERSONNEL'] + '_' + df_existant['KeyDate']
            faits_mission['COMPOSITE_KEY'] = faits_mission['ID_MISSION'] + '_' + faits_mission['ID_PERSONNEL'] + '_' + faits_mission['KeyDate']
            
            # Identifier les nouvelles lignes
            keys_existantes = set(df_existant['COMPOSITE_KEY'])
            nouvelles_lignes = faits_mission[~faits_mission['COMPOSITE_KEY'].isin(keys_existantes)]
            
            if len(nouvelles_lignes) > 0:
                # Supprimer la colonne composite avant la sauvegarde
                nouvelles_lignes = nouvelles_lignes.drop('COMPOSITE_KEY', axis=1)
                df_existant = df_existant.drop('COMPOSITE_KEY', axis=1)
                
                # Concaténer avec les données existantes
                df_maj = pd.concat([df_existant, nouvelles_lignes], ignore_index=True)
                df_maj.to_csv(output_path, index=False)
                print(f"Table de faits MISSION mise à jour avec {len(nouvelles_lignes)} nouvelles lignes pour la date {date_str}")
            else:
                print(f"Aucune nouvelle ligne à ajouter pour la date {date_str}")
        else:
            # Sinon on crée une nouvelle table
            faits_mission.to_csv(output_path, index=False)
            print(f"Table de faits MISSION créée avec {len(faits_mission)} lignes pour la date {date_str}")
            
    except Exception as e:
        print(f"Erreur lors de la création/mise à jour de la table MISSION: {e}")
        # For debugging
        import traceback
        print(traceback.format_exc())

def creer_table_fait_materiel(date_str):
    """Crée ou met à jour la table de faits matériel"""
    output_path = os.path.join(DW_PATH, "FAITS_MATERIEL", "faits_materiel.csv")
    
    # Extraire les données
    try:
        df_materiel = lire_fichiers_source(date_str, "MATERIEL_INFORMATIQUE")
        
        # Créer les dimensions associées
        creer_dimension_date(df_materiel['DATE_PARIS'])
        creer_dimension_materiel(df_materiel)
        
        # Créer la table de faits simplifiée
        df_materiel['KeyDate'] = df_materiel['DATE_PARIS'].dt.strftime('%Y%m%d')
        faits_materiel = df_materiel[['ID_MATERIELINFO', 'ID_PERSONNEL', 'KeyDate']].copy()
        
        # Convertir explicitement toutes les colonnes en string
        faits_materiel['ID_MATERIELINFO'] = faits_materiel['ID_MATERIELINFO'].astype(str)
        faits_materiel['ID_PERSONNEL'] = faits_materiel['ID_PERSONNEL'].astype(str)
        faits_materiel['KeyDate'] = faits_materiel['KeyDate'].astype(str)
        
        # Si la table existe déjà, fusionner avec de nouvelles lignes
        if table_existe(output_path):
            print(f"La table de faits MATERIEL existe déjà, mise à jour...")
            df_existant = pd.read_csv(output_path)
            
            # Convertir explicitement toutes les colonnes en string
            df_existant['ID_MATERIELINFO'] = df_existant['ID_MATERIELINFO'].astype(str)
            df_existant['ID_PERSONNEL'] = df_existant['ID_PERSONNEL'].astype(str)
            df_existant['KeyDate'] = df_existant['KeyDate'].astype(str)
            
            # Créer une clé composite pour identifier les lignes uniques
            df_existant['COMPOSITE_KEY'] = df_existant['ID_MATERIELINFO'] + '_' + df_existant['ID_PERSONNEL'] + '_' + df_existant['KeyDate']
            faits_materiel['COMPOSITE_KEY'] = faits_materiel['ID_MATERIELINFO'] + '_' + faits_materiel['ID_PERSONNEL'] + '_' + faits_materiel['KeyDate']
            
            # Identifier les nouvelles lignes
            keys_existantes = set(df_existant['COMPOSITE_KEY'])
            nouvelles_lignes = faits_materiel[~faits_materiel['COMPOSITE_KEY'].isin(keys_existantes)]
            
            if len(nouvelles_lignes) > 0:
                # Supprimer la colonne composite avant la sauvegarde
                nouvelles_lignes = nouvelles_lignes.drop('COMPOSITE_KEY', axis=1)
                df_existant = df_existant.drop('COMPOSITE_KEY', axis=1)
                
                # Concaténer avec les données existantes
                df_maj = pd.concat([df_existant, nouvelles_lignes], ignore_index=True)
                df_maj.to_csv(output_path, index=False)
                print(f"Table de faits MATERIEL mise à jour avec {len(nouvelles_lignes)} nouvelles lignes pour la date {date_str}")
            else:
                print(f"Aucune nouvelle ligne à ajouter pour la date {date_str}")
        else:
            # Sinon on crée une nouvelle table
            faits_materiel.to_csv(output_path, index=False)
            print(f"Table de faits MATERIEL créée avec {len(faits_materiel)} lignes pour la date {date_str}")
            
    except Exception as e:
        print(f"Erreur lors de la création/mise à jour de la table MATERIEL: {e}")
        # For debugging
        import traceback
        print(traceback.format_exc())

def table_existe(table_path):
    """Vérifie si une table existe déjà"""
    return os.path.exists(table_path)

# Vous devez également mettre à jour la fonction executer_etl_pour_date
def executer_etl_pour_date(date_str):
    """Exécute le processus ETL complet pour une date donnée"""
    print(f"Démarrage de l'ETL pour la date: {date_str}")
    
    try:
        creer_table_fait_mission(date_str)
    except Exception as e:
        print(f"Erreur lors de la mise à jour de la table MISSION: {e}")
    
    try:
        creer_table_fait_materiel(date_str)
    except Exception as e:
        print(f"Erreur lors de la mise à jour de la table MATERIEL: {e}")
    
    print(f"ETL terminé pour la date: {date_str}")


# Option 1: Process the entire year 2024
def process_full_year():
    # Generate all dates in 2024 (leap year, 366 days)
    all_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
    
    # Format dates as YYYYMMDD
    formatted_dates = [date.strftime('%Y%m%d') for date in all_dates]
    
    # Count successful ETL processes
    success_count = 0
    
    # Process each date
    print(f"Starting ETL for all {len(formatted_dates)} days in 2024...")
    for date_str in formatted_dates:
        try:
            print(f"\nProcessing date {date_str} ({success_count+1}/{len(formatted_dates)})")
            executer_etl_pour_date(date_str)
            success_count += 1
        except Exception as e:
            print(f"Failed to process date {date_str}: {e}")
    
    print(f"\nETL completed for {success_count} out of {len(formatted_dates)} days in 2024")

TABLE PERSONNEL
pas besoin de la créer de manière journalière car on pouvait l'extraire directement

In [10]:
#BERLIN
psdf_personnel_BERLIN = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_BERLIN/PERSONNEL_BERLIN.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_BERLIN = psdf_personnel_BERLIN.to_spark(index_col='ID_PERSONNEL')

#LONDON
psdf_personnel_LONDON = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LONDON/PERSONNEL_LONDON.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_LONDON = psdf_personnel_LONDON.to_spark(index_col='ID_PERSONNEL')

#LOSANGELES
psdf_personnel_LOSANGELES = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LOSANGELES/PERSONNEL_LOSANGELES.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_LOSANGELES = psdf_personnel_LOSANGELES.to_spark(index_col='ID_PERSONNEL')

#NEWYORK
psdf_personnel_NEWYORK = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_NEWYORK/PERSONNEL_NEWYORK.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_NEWYORK = psdf_personnel_NEWYORK.to_spark(index_col='ID_PERSONNEL')

#PARIS
psdf_personnel_PARIS = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_PARIS/PERSONNEL_PARIS.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_PARIS = psdf_personnel_PARIS.to_spark(index_col='ID_PERSONNEL')

#SHANGHAI
psdf_personnel_SHANGHAI = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_SHANGHAI/PERSONNEL_SHANGHAI.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_SHANGHAI = psdf_personnel_SHANGHAI.to_spark(index_col='ID_PERSONNEL')
 
 
 
# Code corrigé pour la dimension personnel
import os
from pyspark.sql.functions import lit, col

# Créer le répertoire DIM_PERSONNEL s'il n'existe pas
os.makedirs("DATA_WAREHOUSE/DIM_PERSONNEL", exist_ok=True)

# Ajouter la colonne VILLE pour indiquer la provenance
sdf_personnel_berlin_with_city = sdf_personnel_BERLIN.withColumn("VILLE", lit("BERLIN"))
sdf_personnel_london_with_city = sdf_personnel_LONDON.withColumn("VILLE", lit("LONDON"))
sdf_personnel_la_with_city = sdf_personnel_LOSANGELES.withColumn("VILLE", lit("LOSANGELES"))
sdf_personnel_ny_with_city = sdf_personnel_NEWYORK.withColumn("VILLE", lit("NEWYORK"))
sdf_personnel_paris_with_city = sdf_personnel_PARIS.withColumn("VILLE", lit("PARIS"))
sdf_personnel_shanghai_with_city = sdf_personnel_SHANGHAI.withColumn("VILLE", lit("SHANGHAI"))

# Union de tous les DataFrames avec la colonne VILLE
sdf_personnel_all_with_city = sdf_personnel_berlin_with_city.union(sdf_personnel_london_with_city) \
    .union(sdf_personnel_la_with_city).union(sdf_personnel_ny_with_city) \
    .union(sdf_personnel_paris_with_city).union(sdf_personnel_shanghai_with_city)

sdf_personnel_all_with_city.select("FONCTION_PERSONNEL").distinct().show(truncate=False, n = 1000)
mapping_fonctions = {
    "Ökonom": "Economiste",
    "Führungskraft": "Cadre",
    "Personalleiter": "DRH",
    "Computeringenieur": "Ingénieur Informaticien",
    "Dateningenieur": "Ingénieur Data",
    "Computer Engineer": "Ingénieur Informaticien",
    "HRD": "DRH",
    "Business Executive": "Cadre",
    "Economist": "Economiste",
    "Data Engineer": "Ingénieur Data",
    "Ingénieur Informaticien": "Ingénieur Informaticien",
    "Ingénieur Data": "Ingénieur Data",
    "Economiste": "Economiste",
    "DRH": "DRH",
    "Cadre": "Cadre"
}

sdf_personnel_all_with_city = sdf_personnel_all_with_city.toPandas()

sdf_personnel_all_with_city['FONCTION_PERSONNEL'] = sdf_personnel_all_with_city['FONCTION_PERSONNEL'].map(mapping_fonctions)

sdf_personnel_all_with_city.to_csv('DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv', sep=';', index=False, encoding='utf-8')


+-----------------------+
|FONCTION_PERSONNEL     |
+-----------------------+
|Ökonom                 |
|Führungskraft          |
|Personalleiter         |
|Computeringenieur      |
|Dateningenieur         |
|Computer Engineer      |
|HRD                    |
|Business Executive     |
|Economist              |
|Data Engineer          |
|Ingénieur Informaticien|
|Ingénieur Data         |
|Economiste             |
|DRH                    |
|Cadre                  |
+-----------------------+



EXECUTION de l'ETL boucle avec tous les jours de 2024.

In [11]:
process_full_year()

Starting ETL for all 366 days in 2024...

Processing date 20240101 (1/366)
Démarrage de l'ETL pour la date: 20240101
Erreur lors de la création/mise à jour de la table MISSION: Aucune donnée trouvée pour MISSION à la date 20240101
Traceback (most recent call last):
  File "C:\Users\utcpret\AppData\Local\Temp\ipykernel_17004\1557536114.py", line 217, in creer_table_fait_mission
    df_missions = lire_fichiers_source(date_str, "MISSION")
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\utcpret\AppData\Local\Temp\ipykernel_17004\1557536114.py", line 73, in lire_fichiers_source
    raise FileNotFoundError(f"Aucune donnée trouvée pour {type_donnees} à la date {date_str}")
FileNotFoundError: Aucune donnée trouvée pour MISSION à la date 20240101

Erreur lors de la création/mise à jour de la table MATERIEL: Aucune donnée trouvée pour MATERIEL_INFORMATIQUE à la date 20240101
Traceback (most recent call last):
  File "C:\Users\utcpret\AppData\Local\Temp\ipykernel_1700

C:\Users\utcpret\AppData\Local\Temp\ipykernel_17004\493881328.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coordinates_cache_df = pd.concat([coordinates_cache_df, new_row], ignore_index=True)
C:\Users\utcpret\AppData\Local\Temp\ipykernel_17004\493881328.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  distance_cache_df = pd.concat([distance_cache_df, new_row], ignore_index=True)


mission numero : 1 faite
mission numero : 2 faite
mission numero : 3 faite
mission numero : 4 faite
mission numero : 5 faite
mission numero : 6 faite
mission numero : 7 faite
mission numero : 8 faite
mission numero : 9 faite
mission numero : 10 faite
mission numero : 11 faite
mission numero : 12 faite
mission numero : 13 faite
mission numero : 14 faite
mission numero : 15 faite
mission numero : 16 faite
mission numero : 17 faite
mission numero : 18 faite
mission numero : 19 faite
mission numero : 20 faite
mission numero : 21 faite
mission numero : 22 faite
mission numero : 23 faite
mission numero : 24 faite
mission numero : 25 faite
mission numero : 26 faite
mission numero : 27 faite
mission numero : 28 faite
mission numero : 29 faite
mission numero : 30 faite
mission numero : 31 faite
mission numero : 32 faite
mission numero : 33 faite
mission numero : 34 faite
mission numero : 35 faite
mission numero : 36 faite
mission numero : 37 faite
mission numero : 38 faite
mission numero : 39 f

In [12]:
mission_mapping = {
    "Geschäftstreffen": "Rencontre entreprises",
    "Konferenz": "Conférence",
    "Schulung": "Formation",
    "Entwicklung": "Développement",
    "Meeting": "Réunion",
    "Team Meeting": "Réunion",
    "Business Meeting": "Rencontre entreprises",
    "Conference": "Conférence",
    "Development": "Développement",
    "Vocational Training": "Formation",
    "Conférence": "Conférence",
    "Formation": "Formation",
    "Réunion": "Réunion",
    "Rencontre entreprises": "Rencontre entreprises",
    "Développement": "Développement"
}

# Corrigez la lecture du fichier en utilisant la virgule comme séparateur
pandas_df = pd.read_csv('DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv', sep=',', encoding='utf-8')

# Vérifiez les colonnes après correction
print(pandas_df.columns)

# Maintenant vous pouvez accéder à la colonne TYPE_MISSION correctement
if 'TYPE_MISSION' in pandas_df.columns:
    pandas_df['TYPE_MISSION'] = pandas_df['TYPE_MISSION'].map(mission_mapping).fillna('Autre')
    
    # Sauvegardez le fichier modifié (conservez le même format de séparateur)
    pandas_df.to_csv('DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv', sep=',', index=False, encoding='utf-8')
    
    # Rechargez la vue temporaire
    dim_mission = spark.read.csv("DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv", header=True, inferSchema=True, sep=',')
    dim_mission.createOrReplaceTempView("mission")
    print("Mappings de TYPE_MISSION appliqués avec succès")
else:
    print("La colonne TYPE_MISSION n'existe pas. Voici les colonnes disponibles:", pandas_df.columns)


Index(['ID_MISSION', 'TYPE_MISSION', 'VILLE_DEPART', 'PAYS_DEPART',
       'VILLE_DESTINATION', 'PAYS_DESTINATION', 'TRANSPORT', 'ALLER_RETOUR',
       'GES'],
      dtype='object')
Mappings de TYPE_MISSION appliqués avec succès


Chargement de toutes les tables de dimension et de faits nécessaires pour les analyses

In [13]:
import pyspark.pandas as ps 
from pyspark.sql import functions as F 
from pyspark.sql.types import *

# Lecture du fichier dimension_personnel avec séparateur ";"
dim_personnel = spark.read.csv("DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv", 
                             header=True, 
                             inferSchema=True,
                             sep=";")  # Spécifier le séparateur ";"

# Lecture des autres fichiers avec séparateur "," (valeur par défaut)
dim_materiel = spark.read.csv("DATA_WAREHOUSE/DIM_MATERIEL/dimension_materiel.csv", 
                             header=True, 
                             inferSchema=True)
dim_date = spark.read.csv("DATA_WAREHOUSE/DIM_DATE/dimension_date.csv", 
                         header=True, 
                         inferSchema=True) 
dim_mission = spark.read.csv("DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv", 
                           header=True, 
                           inferSchema=True) 
faits_materiel = spark.read.csv("DATA_WAREHOUSE/FAITS_MATERIEL/faits_materiel.csv", 
                              header=True, 
                              inferSchema=True) 
faits_mission = spark.read.csv("DATA_WAREHOUSE/FAITS_MISSION/faits_mission.csv", 
                             header=True, 
                             inferSchema=True)

# Création des vues temporaires
dim_personnel.createOrReplaceTempView("personnel") 
dim_materiel.createOrReplaceTempView("materiel") 
dim_date.createOrReplaceTempView("dates") 
dim_mission.createOrReplaceTempView("mission") 
faits_materiel.createOrReplaceTempView("faits_materiel") 
faits_mission.createOrReplaceTempView("faits_mission")

print("\nToutes les vues temporaires ont été créées avec succès.")


Toutes les vues temporaires ont été créées avec succès.


Réponse aux questions 1 a 4

In [14]:
# Question 1: Combien d'ingénieurs informaticiens travaillent sur le site de Paris?
q1 = spark.sql("""
    SELECT COUNT(*) as nb_ingenieurs_info_paris
    FROM personnel
    WHERE `FONCTION_PERSONNEL` = 'Ingénieur Informaticien' 
    AND `VILLE` = 'PARIS'
""")
q1.show()

# Question 2: Combien d'ingénieurs Data travaillent sur les sites de London?
q2 = spark.sql("""
    SELECT COUNT(*) as nb_ingenieurs_data_london
    FROM personnel
    WHERE FONCTION_PERSONNEL = 'Ingénieur Data'
    AND VILLE = 'LONDON'
""")
q2.show()

# Question 3: Combien de cadres travaillent dans l'organisation (tous sites compris)?
q3 = spark.sql("""
    SELECT COUNT(*) as nb_cadres_total
    FROM personnel
    WHERE FONCTION_PERSONNEL = 'Cadre'
""")
q3.show()

q4 = spark.sql("""
    SELECT COUNT(DISTINCT f.ID_MATERIELINFO) as nb_pc_portables
    FROM faits_materiel f
    JOIN materiel m ON f.ID_MATERIELINFO = m.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE m.TYPE LIKE '%PC portable%'
    AND d.MOIS BETWEEN 5 AND 10
    AND d.ANNEE = 2024
""")
q4.show()

+------------------------+
|nb_ingenieurs_info_paris|
+------------------------+
|                    1831|
+------------------------+

+-------------------------+
|nb_ingenieurs_data_london|
+-------------------------+
|                     1568|
+-------------------------+

+---------------+
|nb_cadres_total|
+---------------+
|           3955|
+---------------+

+---------------+
|nb_pc_portables|
+---------------+
|           1820|
+---------------+



In [15]:
# Fix Hadoop environment issue for Windows
import os
import sys
import tempfile
import pandas as pd

# Create a temporary directory for Hadoop
hadoop_home = os.path.join(tempfile.gettempdir(), "hadoop")
os.makedirs(os.path.join(hadoop_home, "bin"), exist_ok=True)

# Set Hadoop environment variables
os.environ["HADOOP_HOME"] = hadoop_home
os.environ["hadoop.home.dir"] = hadoop_home
os.environ["PATH"] = os.environ["PATH"] + os.pathsep + os.path.join(hadoop_home, "bin")


# Analyse de l'Impact Carbone

Réponses aux questions d'analyse d'impact carbone avec Spark SQL.

In [16]:
# Question 5: Quelle a été l'impact carbone des PC portables entre mai et octobre 2024?
q5 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_pc_portables
    FROM materiel m
    JOIN faits_materiel f ON m.ID_MATERIELINFO = f.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE m.TYPE LIKE '%PC portable%'
      AND d.MOIS BETWEEN 5 AND 10
      AND d.ANNEE = 2024
""")
print("Question 5: Impact carbone des PC portables (mai-octobre 2024)")
q5.show()

# Question 6: Quelle a été l'impact carbone des PC fixes sans écran achetés par les ingénieurs Data 
# entre mai et septembre 2024 sur les sites de Paris et New-York?
q6 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_pc_fixes
    FROM materiel m
    JOIN faits_materiel f ON m.ID_MATERIELINFO = f.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    JOIN personnel p ON f.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE m.TYPE LIKE '%PC fixe sans ecran%'
      AND p.FONCTION_PERSONNEL = 'Ingénieur Data'
      AND (p.VILLE = 'PARIS' OR p.VILLE = 'NEWYORK')
      AND d.MOIS BETWEEN 5 AND 9
      AND d.ANNEE = 2024
""")
print("Question 6: Impact carbone des PC fixes sans écran (ingénieurs Data, Paris et NY, mai-septembre 2024)")
q6.show()

# Question 7: Quelle a été l'impact carbone des Ecrans achetés par les cadres 
# entre mai et septembre 2024 sur tous les sites de l'organisation?
q7 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_ecrans
    FROM materiel m
    JOIN faits_materiel f ON m.ID_MATERIELINFO = f.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    JOIN personnel p ON f.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE m.TYPE LIKE '%Ecran%'
      AND p.FONCTION_PERSONNEL = 'Cadre'
      AND d.MOIS BETWEEN 5 AND 9
      AND d.ANNEE = 2024
""")
print("Question 7: Impact carbone des écrans achetés par les cadres (mai-septembre 2024)")
q7.show()

# Question 8: Quelle a été l'impact carbone des missions sur les sites Européens
# en juillet 2024?
q8 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_missions_europe
    FROM mission m
    JOIN faits_mission f ON m.ID_MISSION = f.ID_MISSION
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE (m.PAYS_DEPART IN ('France', 'Allemagne', 'England') 
           OR m.PAYS_DESTINATION IN ('France', 'Allemagne', 'England'))
      AND d.MOIS = 7
      AND d.ANNEE = 2024
""")
print("Question 8: Impact carbone des missions sur les sites européens (juillet 2024)")
q8.show()

# Question 9: Quels ont été les 5 jours les plus impactants concernant les missions 
# en avion sur l'ensemble des sites de l'organisation?
q9 = spark.sql("""
    SELECT d.DATE_PARIS, SUM(m.GES) as impact_total
    FROM mission m
    JOIN faits_mission f ON m.ID_MISSION = f.ID_MISSION
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE m.TRANSPORT = 'Avion'
    GROUP BY d.DATE_PARIS
    ORDER BY impact_total DESC
    LIMIT 5
""")
print("Question 9: Les 5 jours les plus impactants pour les missions en avion")
q9.show()

# Question 10: Quelle a été la fonction du personnel qui a eu le plus d'impact concernant 
# les missions et le matériel informatique sur l'ensemble des sites de l'organisation?
q10 = spark.sql("""
    SELECT p.FONCTION_PERSONNEL, 
           SUM(COALESCE(mis.GES, 0) + COALESCE(mat.GES, 0)) as impact_total
    FROM personnel p
    LEFT JOIN faits_mission fm ON p.ID_PERSONNEL = fm.ID_PERSONNEL
    LEFT JOIN mission mis ON fm.ID_MISSION = mis.ID_MISSION
    LEFT JOIN faits_materiel fmat ON p.ID_PERSONNEL = fmat.ID_PERSONNEL
    LEFT JOIN materiel mat ON fmat.ID_MATERIELINFO = mat.ID_MATERIELINFO
    GROUP BY p.FONCTION_PERSONNEL
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 10: Fonction du personnel avec le plus d'impact (missions et matériel)")
q10.show()

# Question 11: Quel site a eu le plus d'impact concernant les missions et le 
# matériel informatique sur l'ensemble des sites de l'organisation?
q11 = spark.sql("""
    SELECT p.VILLE, 
           SUM(COALESCE(mis.GES, 0) + COALESCE(mat.GES, 0)) as impact_total
    FROM personnel p
    LEFT JOIN faits_mission fm ON p.ID_PERSONNEL = fm.ID_PERSONNEL
    LEFT JOIN mission mis ON fm.ID_MISSION = mis.ID_MISSION
    LEFT JOIN faits_materiel fmat ON p.ID_PERSONNEL = fmat.ID_PERSONNEL
    LEFT JOIN materiel mat ON fmat.ID_MATERIELINFO = mat.ID_MATERIELINFO
    GROUP BY p.VILLE
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 11: Site avec le plus d'impact (missions et matériel)")
q11.show()

# Question 12: Quel a été l'impact carbone des missions reliant chaque site
# durant le mois de mai 2024?
q12 = spark.sql("""
    SELECT mis.VILLE_DEPART, mis.VILLE_DESTINATION, SUM(mis.GES) as impact_carbone
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    WHERE mis.VILLE_DEPART IN ('Paris', 'Berlin', 'London', 'Los Angeles', 'New-York', 'Shanghai')
      AND mis.VILLE_DESTINATION IN ('Paris', 'Berlin', 'London', 'Los Angeles', 'New-York', 'Shanghai')
      AND d.MOIS = 5
      AND d.ANNEE = 2024
    GROUP BY mis.VILLE_DEPART, mis.VILLE_DESTINATION
    ORDER BY impact_carbone DESC
""")
print("Question 12: Impact carbone des missions entre sites (mai 2024)")
q12.show(20)  # Show more rows to see all site combinations

# Question 13: Quel a été l'impact carbone des séminaires en juillet 2024 pour 
# les employés de Los Angeles?
q13 = spark.sql("""
    SELECT SUM(mis.GES) as impact_seminaires
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    JOIN personnel p ON fm.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE mis.TYPE_MISSION = 'Conférence'
      AND p.VILLE = 'LOSANGELES'
      AND d.MOIS = 7
      AND d.ANNEE = 2024
""")
print("Question 13: Impact carbone des séminaires (employés de Los Angeles, juillet 2024)")
q13.show()

# Question 14: Quel a secteur d'activité a été le plus impactant pour les missions
# "conférences" entre mai et octobre 2024?
q14 = spark.sql("""
    SELECT p.FONCTION_PERSONNEL, SUM(mis.GES) as impact_total
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    JOIN personnel p ON fm.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE mis.TYPE_MISSION = 'Conférence'
      AND d.MOIS BETWEEN 5 AND 10
      AND d.ANNEE = 2024
    GROUP BY p.FONCTION_PERSONNEL
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 14: Secteur d'activité le plus impactant pour les conférences (mai-octobre 2024)")
q14.show()

# Question 15: Quel a été l'âge moyen des employés qui sont partis en formations
# entre juillet et septembre 2024?
q15 = spark.sql("""
    SELECT AVG(YEAR(CURRENT_DATE()) - YEAR(p.DT_NAISS)) as age_moyen
    FROM personnel p
    JOIN faits_mission fm ON p.ID_PERSONNEL = fm.ID_PERSONNEL
    JOIN mission mis ON fm.ID_MISSION = mis.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    WHERE mis.TYPE_MISSION = 'Formation'
      AND d.MOIS BETWEEN 7 AND 9
      AND d.ANNEE = 2024
""")
print("Question 15: Âge moyen des employés partis en formation (juillet-septembre 2024)")
q15.show()

# Question 16: Quelle destination a été la plus impactante (en cumul) entre mai et octobre 2024?
q16 = spark.sql("""
    SELECT mis.VILLE_DESTINATION, SUM(mis.GES) as impact_total
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    WHERE d.MOIS BETWEEN 5 AND 10
      AND d.ANNEE = 2024
    GROUP BY mis.VILLE_DESTINATION
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 16: Destination la plus impactante (mai-octobre 2024)")
q16.show()

Question 5: Impact carbone des PC portables (mai-octobre 2024)
+---------------------------+
|impact_carbone_pc_portables|
+---------------------------+
|          964.5539999999817|
+---------------------------+

Question 6: Impact carbone des PC fixes sans écran (ingénieurs Data, Paris et NY, mai-septembre 2024)
+-----------------------+
|impact_carbone_pc_fixes|
+-----------------------+
|     123.49600000000005|
+-----------------------+

Question 7: Impact carbone des écrans achetés par les cadres (mai-septembre 2024)
+---------------------+
|impact_carbone_ecrans|
+---------------------+
|    40.88000000000003|
+---------------------+

Question 8: Impact carbone des missions sur les sites européens (juillet 2024)
+------------------------------+
|impact_carbone_missions_europe|
+------------------------------+
|             8543.716438160567|
+------------------------------+

Question 9: Les 5 jours les plus impactants pour les missions en avion
+----------+-----------------+
|DA

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# Exécution de la requête SQL
q18_sql = """
    SELECT
        m.VILLE_DEPART,
        m.VILLE_DESTINATION,
        SUM(m.GES) AS total_GES
    FROM mission m
    JOIN faits_mission fm ON m.ID_MISSION = fm.ID_MISSION
    JOIN personnel p ON fm.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE m.VILLE_DEPART = 'Paris'
    GROUP BY m.VILLE_DEPART, m.VILLE_DESTINATION
    ORDER BY total_GES DESC
    LIMIT 5
"""

q18_result = spark.sql(q18_sql)

# Conversion en DataFrame Pandas
df_q18 = q18_result.toPandas()
df_q18['TRAJET'] = df_q18['VILLE_DEPART'] + " → " + df_q18['VILLE_DESTINATION']

# Création du plot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_q18, x='TRAJET', y='total_GES', palette='Reds_d')
plt.title("TOP 5 trajets les plus impactants depuis Paris")
plt.ylabel("Impact carbone (GES)")
plt.xlabel("Trajet")
plt.xticks(rotation=45)
plt.tight_layout()

# Enregistrement dans un fichier PDF
with PdfPages("Figure_18.pdf") as pdf:
    pdf.savefig()
    plt.close()

C:\Users\utcpret\AppData\Local\Temp\ipykernel_17004\3381429238.py:28: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_q18, x='TRAJET', y='total_GES', palette='Reds_d')


In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

q19_sql = """
    SELECT
        d.ANNEE,
        d.MOIS,
        p.VILLE,
        mis.TRANSPORT,
        SUM(mis.GES) as total_impact
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    JOIN personnel p ON fm.ID_PERSONNEL = p.ID_PERSONNEL
    GROUP BY d.ANNEE, d.MOIS, p.VILLE, mis.TRANSPORT
    ORDER BY d.ANNEE, d.MOIS, p.VILLE, mis.TRANSPORT
"""

q19_result = spark.sql(q19_sql)
df = q19_result.toPandas()

df['PERIODE'] = df['ANNEE'].astype(str) + "-" + df['MOIS'].astype(str).str.zfill(2)

sites = df['VILLE'].unique()
palette = sns.color_palette("tab10", n_colors=len(df['TRANSPORT'].unique()))

with PdfPages("figure_19.pdf") as pdf:
    for site in sites:
        site_df = df[df['VILLE'] == site].copy()
        if not site_df.empty:
            plt.figure(figsize=(14, 8))
            sns.barplot(
                data=site_df,
                x='PERIODE',
                y='total_impact',
                hue='TRANSPORT',
                palette=palette
            )
            plt.yscale('log')
            plt.title(f"Impact carbone mensuel par type de transport - {site}")
            plt.xlabel("Mois")
            plt.ylabel("Impact carbone (GES) [Échelle logarithmique]")
            plt.xticks(rotation=45)
            plt.legend(title="Transport", bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.tight_layout()

            pdf.savefig()  # Sauvegarde dans le PDF
            plt.close()    # Ferme la figure proprement



In [19]:
# Question 20: Proposer une figure illustrant l’impact carbone global mensuel de l’organisation.

q20_sql_material = """
    SELECT
        d.ANNEE,
        d.MOIS,
        SUM(m.GES) as total_impact
    FROM materiel m
    JOIN faits_materiel fm ON m.ID_MATERIELINFO = fm.ID_MATERIELINFO
    JOIN dates d ON fm.KeyDate = d.KeyDate
    GROUP BY d.ANNEE, d.MOIS
"""
q20_result_material = spark.sql(q20_sql_material)

q20_sql_mission = """
    SELECT
        d.ANNEE,
        d.MOIS,
        SUM(mis.GES) as total_impact
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    GROUP BY d.ANNEE, d.MOIS
"""

q20_result_material = spark.sql(q20_sql_material)
df_q20 = q20_result_material.toPandas()
df_q20['PERIODE'] = df_q20['ANNEE'].astype(str) + "-" + df_q20['MOIS'].astype(str).str.zfill(2)
df_q20 = df_q20.sort_values(by='PERIODE')

plt.figure(figsize=(12, 6))
plt.plot(df_q20['PERIODE'], df_q20['total_impact'], marker='o', color='steelblue')
plt.title("Impact carbone global mensuel de l'organisation (matériel)")
plt.xlabel("Mois")
plt.ylabel("Impact carbone total (GES)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("figure_20.pdf")
plt.close()

# STREAMLIT

In [20]:
!pip install -q streamlit
!npm install localtunnel
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pandas as pd
import numpy as np
from datetime import datetime, date
!pip install numpy==1.23.5
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession.builder.getOrCreate()


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\utcpret\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
'npm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.6 MB 3.2 MB/s eta 0:00:05
   ---------------------------------------- 0.1/14.6 MB 1.3 MB/s eta 0:00:12
    --------------------------------------- 0.2/14.6 MB 1.7 MB/s eta 0:00:09
    --------------------------------------- 0.3/14.6 MB 1.7 MB/s eta 0:00:09
   - -------------------------------------- 0.7/14.6 MB 2.8 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/14.6 MB 3.2 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/14.6 MB 3.9 MB/s eta 0:00:04
   ---- ----------------------------------- 1.5/14.6 MB 3.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/14.6 MB 3.6 MB/s eta 0:00:04
   ----- ---------------------------------- 2.0/14.6 MB 4.0 MB/s eta 0:00:04
   ------ --------------------------------- 2.3/14.6 MB 4.3 MB/s eta 0:00:03
   ------- -------------------------------- 2.6/14.6 MB 4.5 MB/s eta 0:00:03
   ---

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\utcpret\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

st.set_page_config(page_title="Dashboard Carbone", layout="wide")
st.title("📊 Dashboard Carbone - Organisation (à partir de dimension_mission.csv)")

@st.cache_data
def load_data():
    df_mission = pd.read_csv("DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv", encoding='utf-8')
    df_faits = pd.read_csv("DATA_WAREHOUSE/FAITS_MISSION/faits_mission.csv", encoding='utf-8')
    df_dates = pd.read_csv("DATA_WAREHOUSE/DIM_DATE/dimension_date.csv", encoding='utf-8')

    df_all = df_faits.merge(df_mission, on="ID_MISSION", how="left")
    df_all = df_all.merge(df_dates, on="KeyDate", how="left")

    df_all['VILLE_DEPART'] = df_all['VILLE_DEPART'].str.upper()
    df_all['VILLE_DESTINATION'] = df_all['VILLE_DESTINATION'].str.upper()
    df_all['TRANSPORT'] = df_all['TRANSPORT'].str.title()
    df_all['TYPE_MISSION'] = df_all['TYPE_MISSION'].str.title()

    return df_all

df = load_data()

ANNEE = 2024
mois_list = list(range(1, 13))
ville_list = sorted(df['VILLE_DEPART'].dropna().unique().tolist())
transport_list = sorted(df['TRANSPORT'].dropna().unique().tolist())
type_mission_list = sorted(df['TYPE_MISSION'].dropna().unique().tolist())

st.title("Dashboard Carbone - Organisation")

question = st.sidebar.selectbox(
    "Choisissez une question à visualiser",
    [
        "Impact carbone mensuel par transport et site (Q19)",
        "Impact carbone global mensuel de l’organisation (Q20)",
        "Impact carbone séminaires Los Angeles (Q13)",
        "Destination la plus impactante (Q16)"
    ]
)

# Q19 - Transport et site
if question == "Impact carbone mensuel par transport et site (Q19)":
    st.header("Impact carbone mensuel par type de transport et site (Q19)")

    mois_sel = st.multiselect("Sélectionnez les mois", mois_list, default=mois_list)
    ville_sel = st.multiselect("Sélectionnez les villes de départ", ville_list, default=ville_list)
    transport_sel = st.multiselect("Sélectionnez les types de transport", transport_list, default=transport_list)

    df_filtered = df[
        (df['ANNEE'] == ANNEE) &
        (df['MOIS'].isin(mois_sel)) &
        (df['VILLE_DEPART'].isin(ville_sel)) &
        (df['TRANSPORT'].isin(transport_sel))
    ].copy()

    if df_filtered.empty:
        st.warning("Aucune donnée disponible pour les filtres sélectionnés.")
    else:
        for ville in ville_sel:
            ville_df = df_filtered[df_filtered['VILLE_DEPART'] == ville].copy()
            if not ville_df.empty:
                plt.figure(figsize=(10, 5))
                sns.barplot(
                    data=ville_df,
                    x="MOIS", y="GES",
                    hue="TRANSPORT",
                    estimator=np.sum,
                    errorbar=None
                )
                plt.yscale("log")
                plt.title(f"Impact carbone (CO2 en tonne) par transport – {ville}")
                plt.xlabel("Mois")
                plt.ylabel("GES (échelle log)")
                st.pyplot(plt.gcf())
                plt.clf()

# Q20 - Global organisation
elif question == "Impact carbone global mensuel de l’organisation (Q20)":
    st.header("Impact carbone global mensuel de l’organisation (Q20)")

    df_filtered = df[df['ANNEE'] == ANNEE].copy()
    df_grouped = df_filtered.groupby('MOIS')['GES'].sum().reindex(mois_list, fill_value=0).reset_index()

    plt.figure(figsize=(10, 4))
    sns.lineplot(data=df_grouped, x='MOIS', y='GES', marker='o')
    plt.title("Impact carbone global mensuel (CO2 en tonne)")
    plt.xlabel("Mois")
    plt.ylabel("GES")
    st.pyplot(plt.gcf())
    plt.clf()

# Q13 - Séminaires selon ville de départ
elif question == "Impact carbone séminaires Los Angeles (Q13)":
    st.header("Impact carbone des séminaires organisés par ville de départ (Q13)")

    villes_possible = ['BERLIN', 'LONDON', 'LOS ANGELES', 'NEW-YORK', 'PARIS', 'SHANGHAI']
    ville_sel = st.selectbox("Sélectionnez une ville de départ", villes_possible, index=villes_possible.index("LOS ANGELES"))
    types_possible = ['Conférence', 'Développement', 'Formation', 'Réunion', 'Rencontre entreprises']
    type_sel = st.selectbox("Sélectionnez une ville de départ", types_possible, index=types_possible.index("Conférence"))
    mois_sel = st.multiselect("Sélectionnez les mois", mois_list, default=[7])

    df_filtered = df[
        (df['ANNEE'] == ANNEE) &
        (df['MOIS'].isin(mois_sel)) &
        (df['VILLE_DEPART'] == ville_sel) &
        (df['TYPE_MISSION'] == type_sel)
    ].copy()

    if df_filtered.empty:
        st.warning(f"Aucune mission trouvée depuis {ville_sel} sur les mois sélectionnés.")
    else:
        impact = df_filtered['GES'].sum()
        st.success(
            f"Impact carbone total des séminaires partant de **{ville_sel}** : "
            f"**{impact:.2f} GES**"
        )

# Q16 - Destination la plus impactante (filtrable)
elif question == "Destination la plus impactante (Q16)":
    st.header("Destination la plus impactante (Q16)")

    mois_sel = st.multiselect("Sélectionnez les mois", mois_list, default=list(range(5, 11)))

    df_filtered = df[
        (df['ANNEE'] == 2024) &
        (df['MOIS'].isin(mois_sel))
    ].copy()

    if df_filtered.empty:
        st.warning("Aucune donnée disponible pour les mois sélectionnés.")
    else:
        df_grouped = df_filtered.groupby("VILLE_DESTINATION")['GES'].sum().reset_index()
        top_dest = df_grouped.loc[df_grouped['GES'].idxmax()]

        st.success(
            f"Destination la plus impactante pour {', '.join(map(str, mois_sel))} : "
            f"**{top_dest['VILLE_DESTINATION']}** avec **{top_dest['GES']:.2f} GES**."
        )

Overwriting app.py


In [22]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

& �tait inattendu.
